In [10]:
import pandas as pd
import numpy as np
import os
import itertools

In [11]:
os.chdir('c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL')
os.getcwd()

'c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL'

In [12]:
scores = pd.read_csv('backend/data/games/scores.csv')
scores['date'] = pd.to_datetime(scores['date'])
scores['first_score_time'] = scores['first_score_time'].apply(lambda x: int(x.split(':')[0]) + round(float(x.split(':')[1]) / 60, 2))
scores['first_score_time'] = scores['first_score_time'] + np.where(scores['first_score_quarter'] == 1, 0, 15)
scores['first_score'] = scores['team'].apply(lambda x: x.split(' ')[-1]) == scores['first_score_team']
scores = scores[['date', 'team', 'opponent', 'week', 'season', 'first_score_team', 'first_score_quarter', 'first_score_time', 'first_score', 'home_field']]
scores.tail(5)

,date,team,opponent,week,season,first_score_team,first_score_quarter,first_score_time,first_score,home_field
6539,2022-09-25,Tampa Bay Buccaneers,Green Bay Packers,3,2022,Buccaneers,1,9.08,True,True
6540,2022-09-25,San Francisco 49ers,Denver Broncos,3,2022,49ers,1,7.93,True,False
6541,2022-09-25,Denver Broncos,San Francisco 49ers,3,2022,49ers,1,7.93,False,True
6542,2022-09-26,Dallas Cowboys,New York Giants,3,2022,Cowboys,1,2.15,True,False
6543,2022-09-26,New York Giants,Dallas Cowboys,3,2022,Cowboys,1,2.15,False,True


In [13]:
df = scores.groupby(['home_field', 'first_score']).agg({'first_score': 'count'})
df = df.join(scores.groupby(['home_field'])['week'].count(), how='inner')
df['accuracy'] = round(df['first_score'] / df['week'], 2)
df.head(10)

first_score  week  accuracy
home_field first_score                             
False      False               1815  3272      0.55
           True                1457  3272      0.45
True       False               1468  3272      0.45
           True                1804  3272      0.55

In [14]:
df = scores[(scores['season'] >= 2021) & (scores['week'] >= 4)].groupby(['team', 'first_score']).agg({'first_score_time': 'mean', 'first_score': 'count'})
df = df.join(scores[scores['season'] == 2021].groupby(['team'])['week'].count(), how='inner')
df['accuracy'] = round(df['first_score'] / df['week'], 2)
df.tail(15)

first_score_time  first_score  week  \
team                     first_score                                        
New York Jets            False                9.150000            9    17   
                         True                 9.200000            5    17   
Philadelphia Eagles      False               10.857778            9    18   
                         True                 8.835000            6    18   
Pittsburgh Steelers      False                9.428333            6    18   
                         True                14.711111            9    18   
San Francisco 49ers      False               10.488000           10    20   
                         True                 7.295714            7    20   
Seattle Seahawks         False               12.372500            8    17   
                         True                11.078333            6    17   
Tampa Bay Buccaneers     False                9.277500            8    19   
                         True                 9.230000            8    19   
Tennessee Titans         False                7.465714            7    18   
                         True                 9.325000            8    18   
Washington Football Team False                9.990000           14    17   

                                      accuracy  
team                     first_score            
New York Jets            False            0.53  
                         True             0.29  
Philadelphia Eagles      False            0.50  
                         True             0.33  
Pittsburgh Steelers      False            0.33  
                         True             0.50  
San Francisco 49ers      False            0.50  
                         True             0.35  
Seattle Seahawks         False            0.47  
                         True             0.35  
Tampa Bay Buccaneers     False            0.42  
                         True             0.42  
Tennessee Titans         False            0.39  
                         True             0.44  
Washington Football Team False            0.82

In [15]:
teams = ('Miami Dolphins', 'Cincinnati Bengals')
first_score = (True, False)
away_first = df.loc[(teams[0], first_score[0]), :]['accuracy'] * df.loc[(teams[1], first_score[1]), :]['accuracy']
home_first = df.loc[(teams[0], first_score[1]), :]['accuracy'] * df.loc[(teams[1], first_score[0]), :]['accuracy']

In [16]:
round(away_first / (away_first + home_first) * 100, 1)

74.2

In [17]:
round(home_first / (away_first + home_first) * 100, 1)

25.8